This is to scrap lyrics from the website for wf's lyrics.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [21]:
# URL of the main page
url = "https://mojim.com/cnh100019.htm"

# Send a GET request
response = requests.get(url)

# Parse the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'dd' elements with class 'hb2'
dd_elements = soup.find_all('dd', class_='hb2')

# Initialize empty lists to store the URLs, song names, and album names
urls = []
song_names = []
album_names = []

# Iterate over each 'dd' element
for dd in dd_elements:
    # Get the album name from the 'hc1' span
    album_name = dd.find('span', class_='hc1').get_text(strip=True)
    
    # Find all 'a' elements inside the 'dd' element
    a_elements = dd.find_all('a')
    
    # Iterate over each 'a' element
    for a in a_elements:
        # Get the 'href' attribute of the 'a' element, which is the URL
        song_url = a['href']
        
        # Add the base URL if the URL is relative
        if song_url.startswith('/'):
            song_url = "https://mojim.com" + song_url
        
        # Append the URL to the list
        urls.append(song_url)
        
        # Get the song name from the 'title' attribute and remove the " 歌词" part
        song_name = a.get('title').replace(' 歌词', '')
        
        # Append the song name and album name to their respective lists
        song_names.append(song_name)
        album_names.append(album_name)

# Print the URLs, song names, and album names
for url, song_name, album_name in zip(urls, song_names, album_names):
    print(f"URL: {url}, Song Name: {song_name}, Album Name: {album_name}")


URL: https://mojim.com/cn100019x82.htm, Song Name: 归途有风 王菲( Faye Wong ), Album Name: 归途有风
URL: https://mojim.com/cny100019x82x1.htm, Song Name: 归途有风, Album Name: 归途有风
URL: https://mojim.com/cn100019x81.htm, Song Name: 如愿 王菲( Faye Wong ), Album Name: 如愿
URL: https://mojim.com/cny100019x81x1.htm, Song Name: 如愿, Album Name: 如愿
URL: https://mojim.com/cn100019x79.htm, Song Name: 电影 邪不压正 王菲( Faye Wong ), Album Name: 电影 邪不压正
URL: https://mojim.com/cny100019x79x1.htm, Song Name: 偶遇, Album Name: 电影 邪不压正
URL: https://mojim.com/cn100019x77.htm, Song Name: 电影《无问西东》推广曲 王菲( Faye Wong ), Album Name: 电影《无问西东》推广曲
URL: https://mojim.com/cny100019x77x1.htm, Song Name: 无问西东, Album Name: 电影《无问西东》推广曲
URL: https://mojim.com/cn100019x75.htm, Song Name: 电影《从你的全世界路过》片尾曲 王菲( Faye Wong ), Album Name: 电影《从你的全世界路过》片尾曲
URL: https://mojim.com/cny100019x75x1.htm, Song Name: 你在终点等我, Album Name: 电影《从你的全世界路过》片尾曲
URL: https://mojim.com/cn100019x73.htm, Song Name: 敷衍 王菲( Faye Wong ), Album Name: 敷衍
URL: https://mojim.com/c

get the raw lyrics content. 

In [22]:

def get_lyrics(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dl_element = soup.find('dl', id='fsZx1')
    lyrics = ''
    # Check if the 'dl' element was found
    if dl_element is not None:
        lyrics = dl_element.get_text(separator='\n')
    return lyrics


In [23]:
# Initialize an empty list to store the lyrics
lyrics_list = []


# For each URL in the list, get the lyrics.
for url in urls:
    lyrics = get_lyrics(url)
    lyrics_list.append(lyrics)

# Now you have a list of lyrics corresponding to each URL.


In [27]:
df = pd.DataFrame({
    'URL': urls,
    'Lyrics': lyrics_list,
    'Title': song_names,
    'Album Title': album_names
})

# Drop rows where 'Lyrics' is empty or null
df = df[df['Lyrics'].astype(bool)]

# Assuming you have a DataFrame named df
df.to_csv('lyrics_wf.csv', index=False)


Another version: organize the song name, album name, and the composers, clean up the lyrics content...and the spotify song link? as raw data. 
I only get song names right, still working on it. 

In [16]:
import requests
from bs4 import BeautifulSoup

# Initialize the lists
song_names = []
composers = []
lyricists = []
arrangers = []
urls = []

# For each URL in the list
for url in urls:
    # Send a GET request
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the 'dl' element with id 'fsZx1'
    song_element = soup.find('dl', id='fsZx1')

    # Get the text content of the 'dl' element
    song_details = song_element.get_text(separator='\n')

    # Split the text content by newlines
    lines = song_details.split('\n')

    # Initialize the variables
    song_name = composer = lyricist = arranger = lyrics = ''

    # Find the song details
    for line in lines:
        if '作词：' in line:
            lyricist = line.split('作词：', 1)[1]
        elif '作曲：' in line:
            composer = line.split('作曲：', 1)[1]
        elif '编曲：' in line:
            arranger = line.split('编曲：', 1)[1]
        elif line:
            if not song_name:
                song_name = line
            elif not (composer or lyricist or arranger):
                lyrics += line + '\n'

    # Append the details to the lists
    song_names.append(song_name)
    composers.append(composer)
    lyricists.append(lyricist)
    arrangers.append(arranger)
    lyrics_list.append(lyrics)

# Print the lists
print('Song Names:', song_names)
print('Composers:', composers)
print('Lyricists:', lyricists)
print('Arrangers:', arrangers)
print('Lyrics:', lyrics_list)
